# d

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!ls gdrive/MyDrive/datasets/ham10000

HAM10000_images_part_1	HAM10000_images_part_2	HAM10000_metadata.csv


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(0)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
import torch
import torchvision.datasets
from torchvision import transforms

In [ ]:
X_train = torch.load('gdrive/MyDrive/datasets/ham_100_X_train.pt')
y_train = torch.load('gdrive/MyDrive/datasets/ham_100_y_train.pt')
X_test = torch.load('gdrive/MyDrive/datasets/ham_100_X_test.pt')
y_test = torch.load('gdrive/MyDrive/datasets/ham_100_y_test.pt')

In [ ]:
X_train = torch.nn.functional.normalize(X_train)

In [ ]:
X_test = torch.nn.functional.normalize(X_test)

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 100, 100, 3]), torch.Size([8012]))

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 100, 100, 3]), torch.Size([8012]))

In [ ]:
X_train = X_train.permute(0, 3, 1, 2)
X_test = X_test.permute(0, 3, 1, 2)

In [ ]:
import random
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
import sys
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks')

In [ ]:
import pytorch_optimizer as optim

In [ ]:
def train(net, X_train, y_train, X_test, y_test):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    net = net.to(device)
    loss = torch.nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(net.parameters(), lr=5.0e-2)
    optimizer = torch.optim.Adam(net.parameters(), lr=0.5e-3)
    #optimizer = optim.PNMBelief(net.parameters(), lr=0.001)
    #optimizer = optim.YogiPNM(net.parameters())
    #optimizer = optim.AdaBelief(net.parameters())
    #optimizer = optim.DiffGrad(net.parameters())
    #optimizer = optim.Yogi(net.parameters())
    
    batch_size = 10

    test_accuracy_history = []
    test_loss_history = []

    X_test = X_test.to(device)
    y_test = y_test.to(device)

    for epoch in range(14):
        print("epoch ", epoch)
        order = np.random.permutation(len(X_train))
        
        if epoch == 9:
            optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-5)
            #optimizer = optim.PNMBelief(net.parameters(), lr=1.0e-4)
            #optimizer = optim.YogiPNM(net.parameters(), lr=1.0e-4)
            #optimizer = optim.AdaBelief(net.parameters(), lr=1.0e-4)
            #optimizer = optim.DiffGrad(net.parameters(), lr=1.0e-4)
            #optimizer = optim.Yogi(net.parameters(), lr=1.0e-4)
    

        for start_index in tqdm(range(0, len(X_train), batch_size)):
            optimizer.zero_grad()
            net.train()

            batch_indexes = order[start_index:start_index+batch_size]

            X_batch = X_train[batch_indexes].to(device)
            y_batch = y_train[batch_indexes].to(device)

            preds = net.forward(X_batch)

            loss_value = loss(preds, y_batch)
            loss_value.backward()

            optimizer.step()
            
            X_batch

        net.eval()
        with torch.no_grad():
            test_preds = net.forward(X_test)
            loss_value = loss(test_preds, y_test).item()
            test_loss_history.append(loss_value)

            accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().item()
            test_accuracy_history.append(accuracy)
            #print(test_preds.argmax(dim=1))
            #print(y_test)

        print("accuracy = ", accuracy)
        print("\n\n")

    return test_accuracy_history, test_loss_history

In [ ]:
accuracies = {}
losses = {}

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

In [ ]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
model.fc = torch.nn.Linear(model.fc.in_features, 7)

In [ ]:
accuracies['resnet50+Adam'], losses['resnet50+Adam'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [00:34<00:00, 23.38it/s]


accuracy =  0.7638542056083679



epoch  1


100%|██████████| 802/802 [00:33<00:00, 23.91it/s]


accuracy =  0.7613579630851746



epoch  2


...

accuracy =  0.8162755966186523



epoch  12


100%|██████████| 802/802 [00:33<00:00, 23.88it/s]


accuracy =  0.8192710876464844



epoch  13


100%|██████████| 802/802 [00:33<00:00, 23.63it/s]


accuracy =  0.8227658271789551





In [ ]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
accuracies['resnet50+PNMBelief'], losses['resnet50+PNMBelief'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [00:46<00:00, 17.40it/s]


accuracy =  0.7933100461959839



epoch  1


100%|██████████| 802/802 [00:45<00:00, 17.55it/s]


accuracy =  0.7903145551681519



epoch  2


...

accuracy =  0.8442336320877075



epoch  12


100%|██████████| 802/802 [00:45<00:00, 17.71it/s]


accuracy =  0.8442336320877075



epoch  13


100%|██████████| 802/802 [00:46<00:00, 17.40it/s]


accuracy =  0.8532201647758484





In [ ]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
accuracies['resnet50+Yogi'], losses['resnet50+Yogi'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [00:37<00:00, 21.59it/s]


accuracy =  0.7568647265434265



epoch  1


100%|██████████| 802/802 [00:37<00:00, 21.55it/s]


accuracy =  0.785821259021759



epoch  2


...

accuracy =  0.8277583718299866



epoch  12


100%|██████████| 802/802 [00:37<00:00, 21.51it/s]


accuracy =  0.831752359867096



epoch  13


100%|██████████| 802/802 [00:37<00:00, 21.53it/s]


accuracy =  0.8327508568763733





In [ ]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
accuracies['resnet50+DiffGrad'], losses['resnet50+DiffGrad'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [00:49<00:00, 16.12it/s]


accuracy =  0.7443834543228149



epoch  1


100%|██████████| 802/802 [00:49<00:00, 16.35it/s]


accuracy =  0.7703444957733154



epoch  2


...

accuracy =  0.8217673301696777



epoch  12


100%|██████████| 802/802 [00:48<00:00, 16.44it/s]


accuracy =  0.8267598748207092



epoch  13


100%|██████████| 802/802 [00:48<00:00, 16.67it/s]


accuracy =  0.8232650756835938





# ansamble

In [ ]:
APreds = pd.DataFrame()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
test_preds = []
model.eval()
with torch.no_grad():
            test_preds = model.forward(X_test)

In [ ]:
APreds['Adam'] = test_preds.argmax(dim=1).tolist()

In [ ]:
APreds['PNMBelief'] = test_preds.argmax(dim=1).tolist()

In [ ]:
APreds['Yogi'] = test_preds.argmax(dim=1).tolist()

In [ ]:
APreds['DiffGrad'] = test_preds.argmax(dim=1).tolist()

In [ ]:
APreds

,Adam,PNMBelief,Yogi,DiffGrad
0,4,4,4,4
1,4,4,4,4
2,4,4,4,4
3,0,4,4,2
4,4,4,4,4
...,...,...,...,...
1998,4,4,4,4
1999,4,5,4,4
2000,4,4,4,4
2001,4,4,4,4


In [ ]:
APreds = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_v1.csv')
APreds = APreds.drop(['Unnamed: 0'], axis=1)

In [ ]:
APreds.to_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_v1.csv')

In [ ]:
ARes = pd.DataFrame(np.zeros((2003,7), dtype=int))

for i in range(2003):
    for j in range(4):
        if j == 0:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.8227
        if j == 1:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.8532
        if j == 2:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.817274
        if j == 3:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.8232

In [ ]:
(torch.tensor(ARes.idxmax(axis=1)).to(device) == y_test).float().mean().item()

0.8607089519500732

In [ ]:
data = np.zeros((7,7), dtype=int)
D = pd.DataFrame(data)

for x, y in zip(test_preds.argmax(dim=1).tolist(), y_test.tolist()):
    D[y][x] += 1

In [ ]:
D #Adam

In [ ]:
D #PNMBelief

In [ ]:
D #Yogi

In [ ]:
D #DiffGrad

In [ ]:
data = np.zeros((7,7), dtype=int)
D = pd.DataFrame(data)

for x, y in zip(ARes.idxmax(axis=1).to_list(), y_test.tolist()):
    D[y][x] += 1

In [ ]:
D #ansamble

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
precision_score(y_test, test_preds.argmax(dim=1), average='micro')


In [ ]:
print('precision: ', precision_score(y_test, test_preds.argmax(dim=1), average=None))
print('recall: ', recall_score(y_test, test_preds.argmax(dim=1), average=None))
print('f1_score: ', f1_score(y_test, test_preds.argmax(dim=1), average=None))

# SoftMax

In [ ]:
ARes2 = pd.DataFrame(np.zeros((2003,7), dtype=int))

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
test_preds = []
model.eval()
with torch.no_grad():
            test_preds = model.forward(X_test)

In [ ]:
m = torch.nn.Softmax(dim=1)

In [ ]:
test_preds = m(test_preds) * 0.7958 #Adam

In [ ]:
test_preds = m(test_preds) * 0.8532 #PNMBelief

In [ ]:
test_preds = m(test_preds) * 0.8327 #Yogi

In [ ]:
test_preds = m(test_preds) * 0.8257 #DiffGrad

In [ ]:
ARes2 += pd.DataFrame(test_preds.to('cpu'))

In [ ]:
test_preds

In [ ]:
ARes2

,0,1,2,3,4,5,6
0,8.283680e-08,2.843920e-08,2.437177e-07,5.461397e-07,4.898994,0.000002,2.165003e-06
1,2.120959e-06,2.425432e-06,4.605553e-01,4.087035e-07,4.433564,0.004871,3.206450e-06
2,3.616196e-08,9.099297e-08,5.344520e-07,3.779186e-09,4.895886,0.003113,2.636696e-08
3,7.872387e-01,1.045310e-04,9.681472e-01,4.111417e-03,3.135901,0.002953,8.184277e-05
4,1.279136e+00,5.436795e-01,1.902800e-01,7.635349e-03,2.865139,0.000372,1.232990e-02
...,...,...,...,...,...,...,...
1998,2.850618e-08,4.344698e-08,2.511865e-05,7.743007e-08,4.897962,0.001010,2.499295e-06
1999,8.272499e-06,6.127643e-05,4.285587e-04,1.981346e-06,3.881052,1.017434,4.485057e-06
2000,3.993706e-04,5.013336e-04,8.885592e-03,8.913932e-04,4.886929,0.000131,1.261093e-03
2001,3.663139e-06,2.919798e-06,4.283958e-03,3.597175e-06,4.868225,0.026476,4.812310e-06


In [ ]:
(torch.tensor(ARes2.idxmax(axis=1)).to(device) == y_test).float().mean().item()

0.8532201647758484